# Import Dependencies

In [ ]:
import gensim.downloader as api
from gensim import utils
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import download
from google.colab import drive
import os
from tqdm import tqdm

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#os.chdir(os.getcwd() + '/drive/MyDrive/Colab Notebooks')

# Load GloVe and Data

In [ ]:
abstracts_df = pd.read_csv('abstracts.csv')

In [ ]:
vector_size = 300
model = api.load(f'glove-wiki-gigaword-{vector_size}')

[==================================================] 100.0% 376.1/376.1MB downloaded


# Pre-process Data

In [ ]:
download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
class MyCorpus:

  def __iter__(self):
    abstracts_df.dropna(subset=['description'], inplace=True)
    abstracts_df.reset_index(drop=True, inplace=True)
    for _, row in abstracts_df.iterrows():
      words = utils.simple_preprocess(row['description'])
      # Remove stop words
      words = [word for word in words if word not in stop_words]
      yield words

In [ ]:
docs = MyCorpus()

# Calculate Document Embeddings

In [ ]:
def get_doc_embedding(glove, doc):
  doc = [word for word in doc if word in glove.vocab]
  return np.mean(glove[doc], axis=0)

In [ ]:
num_docs = 767968
not_valid = []
to_save = np.empty((vector_size, num_docs))

for i, doc in enumerate(docs):
  try:
    to_save[:,i] = get_doc_embedding(model, doc)
  except:
    not_valid.append(i)
    to_save[:,i] = np.zeros(vector_size)

In [ ]:
np.save(f'glove_pretrained_{vector_size}', to_save)